In [ ]:


import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
from math import isnan
import pandas as pd
from plotly_resampler import FigureResampler
import os
from datetime import datetime
import plotly.express as px
from tqdm import tqdm
import pickle
from copy import deepcopy
from utils import unit_convert
import statistics




def plot_ecd_cutomized_lei_label_ecd(diffs, sample_rate = 5):

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            mode = 'markers',
            marker = dict(
                size = 1
            ),
            x=diffs['time_key'], 
            y=diffs['ecd'], 
            name="difference ecd at bit",
            yaxis='y'),
    )

    fig.update_layout(
        title = fr'{well_name} - RT CHE - ECD at bit',
        )

    # fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
    # fig.write_html(os.path.join(r"C:\NotOneDrive\Data\for_hci", fr"{well_name}_TransientHydraulics_ECD_{time_stamp_str}.html"))
    fig.show()

def get_latest_ecd(time_key, ecds, threshold = 5):
    for i, ecd in ecds.iterrows():
        for i in range(threshold):
            s = (time_key.replace(tzinfo=None) - ecd['time key'].replace(tzinfo=None)).total_seconds()
            if abs(s) <= i:
                return ecd['ecd at bit']
            # print(time_key.replace(tzinfo=None) ,ecd['time key'].replace(tzinfo=None))

def get_ecd_diff(measure_ecds: pd.DataFrame, model_ecds):
    assert type(measure_ecds['time key']) == type(model_ecds['time key'])

    for i, m in tqdm(measure_ecds.iterrows()):
        get_latest_ecd(m['time key'], model_ecds)


save_dir=r"C:\NotOneDrive\Data\ECD_diff_probability"
channel_ecd_path = r"C:\NotOneDrive\Data\algo_data_nas_copy\BDC 4-02\RawData\Channel_ECD_ESD.csv"
measure_ecds = pd.read_csv(channel_ecd_path, skiprows=[1]).rename(
    columns={
        'Time': 'time key',
        'ECD_ARC_RT': 'm ecd at bit'
    }
)[['time key', 'm ecd at bit']].dropna()
measure_ecds['m ecd at bit'] *= unit_convert('g/cm3', 'kg/m3')
measure_ecds['time key'] = pd.to_datetime(measure_ecds['time key'], utc=False)
# measure_ecds = measure_ecds[measure_ecds['m ecd at bit'] < np.percentile(measure_ecds['m ecd at bit'],95)]
# measure_ecds = measure_ecds[measure_ecds['m ecd at bit'] > np.percentile(measure_ecds['m ecd at bit'],5)]

well_name = 'BDC-4-02_bha_run1_correct_mud_TransientHydraulics'
time_stamp_str = '20240805165848'
data_base_dir = rf'C:\NotOneDrive\Data\algo_seperate_data'
output_base_dir = r'C:\NotOneDrive\Data\merged_input_output_channels'
engine_output_channels_path = os.path.join(output_base_dir, f'{well_name}' + (f'_{time_stamp_str}' if time_stamp_str else '') + '.csv')

channels_path = r"C:\NotOneDrive\Data\algo_seperate_data\BDC-4-02_bha_run1\Channels.csv"
algo_inputs = pd.read_csv(channels_path, skiprows=[1])
algo_inputs = algo_inputs.rename(columns={
    'Time': 'time key',
    'DRILLING_VFLOW': 'flowrate',
    'DEPTH_BD': 'bit depth',
    'STANDPIPE_PRS': 'm_spp',
    'HOOK_LOAD': 'hookload',
    'Surface.RPM': 'rpm'
})
algo_inputs['time key'] = pd.to_datetime(algo_inputs['time key'], utc=True)

algo_outputs = pd.read_csv(engine_output_channels_path, skiprows=[1])
algo_outputs[['ecd at bit']] = algo_outputs[['ecd at bit']].apply(pd.to_numeric)
algo_outputs[['simulated spp']] = algo_outputs[['simulated spp']].apply(pd.to_numeric)
algo_outputs[['drill string pressure loss']] = algo_outputs[['drill string pressure loss']].apply(pd.to_numeric)
algo_outputs['time key'] = pd.to_datetime(algo_outputs['time key'], utc=False)


measure_ecds = measure_ecds[measure_ecds['time key'] < algo_inputs['time key'].values[-1]]
measure_ecds = measure_ecds[measure_ecds['time key'] > algo_inputs['time key'].values[0]]

measure_ecds['time key'] = pd.to_datetime(measure_ecds['time key'], utc=True)

model_ecds = algo_outputs[['time key', 'ecd at bit']].dropna()
merged_ecds = pd.merge(model_ecds, measure_ecds, on='time key', how='outer')

flowrates = algo_inputs[['time key', 'flowrate']]
flowrates['flowrate'] = flowrates['flowrate'].interpolate('nearest')

merged_ecds = pd.merge(merged_ecds, flowrates, on='time key', how='outer')
merged_ecds = merged_ecds.dropna(subset=['ecd at bit'])
merged_ecds.rename(columns={'m ecd at bit': 'measure ecd'}).to_csv(os.path.join(save_dir,"ecd_diff.csv"), index=False)

In [ ]:

merged_ecds['ecd at bit'] = pd.to_numeric(merged_ecds['ecd at bit'])
full_time_keys_flowrate = merged_ecds[['time key', 'flowrate']]

diffs = {
    'time_key': [],
    'ecd': [],
}

flag = 0
for i, merged in tqdm(merged_ecds.iterrows()):
    if pd.isnull(merged['m ecd at bit']): continue
    # if (abs(merged['flowrate']-0) < 1e-8): continue

    diff = merged['m ecd at bit'] - merged['ecd at bit']

    diffs['time_key'].append(merged['time key'])
    diffs['ecd'].append(diff)

diffs_origin = pd.DataFrame(diffs)
print(len(diffs_origin))

In [ ]:

''' filter wrong data '''


q1, q3 = np.quantile(diffs_origin['ecd'], [0.05, 0.95])
print(q1, q3)

abnormal_filter = (diffs_origin['ecd'] > q1) & (diffs_origin['ecd'] < q3)
diffs_filtered = diffs_origin[abnormal_filter]
plot_ecd_cutomized_lei_label_ecd(diffs_filtered)

In [ ]:
''' shift data by mean of normal values '''

upper_limit = np.percentile(diffs_filtered['ecd'], 60)
below_limit = np.percentile(diffs_filtered['ecd'], 10)
system_error = diffs_filtered[(diffs_filtered['ecd'] >= below_limit) & (diffs_filtered['ecd'] < upper_limit)]['ecd'].mean()
print(system_error)


In [ ]:

''' decay indicator '''

diffs_shift = diffs_filtered.copy()
diffs_shift['ecd'] -= system_error
diffs_shift['ecd'] /= system_error 
# diffs_shift['ecd'] /= 0.5
diffs_shift.loc[diffs_shift['ecd']<0, 'ecd'] = 0
# plot_ecd_cutomized_lei_label_ecd(diffs_shift)

# print(len(full_time_keys_flowrate), len(diffs_shift))
full_time_keys_flowrate.rename(columns={'time key': 'time_key'}, inplace=True)
diffs_indicator = full_time_keys_flowrate.set_index('time_key').join(diffs_shift.set_index('time_key'))
# diffs_indicator.loc[diffs_indicator['flowrate'] == 0, 'ecd'] *= 2
# diffs_indicator.loc[pd.isna(diffs_indicator['ecd']), 'ecd'] = 0
diffs_indicator.reset_index(inplace=True)
lbd = 0.99
for i, r in tqdm(diffs_indicator.iterrows()):
    if i == 0:
        continue
    if pd.isna(diffs_indicator['ecd'].loc[i]):
        if diffs_indicator['flowrate'].loc[i] == 0.0:
            diffs_indicator['ecd'].loc[i] = diffs_indicator['ecd'].loc[i-1]
        else:
            diffs_indicator['ecd'].loc[i] = diffs_indicator['ecd'].loc[i-1] * lbd


diffs_indicator['ecd'] = diffs_indicator['ecd'].ewm(halflife='2 h', times = pd.DatetimeIndex(diffs_indicator['time_key'])).mean()
plot_ecd_cutomized_lei_label_ecd(diffs_indicator)
diffs_indicator.rename(columns={"ecd": "probability"})[['time_key', 'probability']].to_csv(os.path.join(save_dir, 'ecd_diff_probability.csv'), index = False)
# diffs_indicator.to_csv("test_ecd_diff_indicator.csv", index=False